In [74]:
import numpy as np
import random

In [513]:
# Perceptron
class Perceptron(object):    
    def __init__(self, n_weights = 3, rate=0.1):
        
        # Neural network variables
        self.n_weights = n_weights # number of weights
        self.rate = rate # learning rate 
        
        # Set initial random weights for the network
        W = np.array([random.uniform(-1,1) for _ in range(self.n_weights)])
        #W = np.zeros(self.n_weights)
        W = np.insert(W, 0, 1, axis=0) # inserting bias
        self.W = W.reshape((-1,1))  
                
    
    # Activation function
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    # Activation function derivative
    def d_sigmoid(self, x): # derivate of sigmoid
        return self.sigmoid(x)*(1 - self.sigmoid(x))
    
    def V(self, X):
        return np.diag(self.d_sigmoid(X @ self.W).flatten())
    
    def H(self, X):
        return X.T @ self.V(X) @ X

    
    # Binary Cross-Entropy loss function
    def loss(self, Z, Y):
        return np.asscalar((-1/len(Y)) * ( Y.T @ np.log(Z + (1.e-10)) + (1 - Y).T @ np.log(1 - Z + (1.e-10)) )) 

    # Gradient of loss function
    def gradient_loss(self, Z, Y):
        dL = Y.T @ (1/Z) + (1 - Y).T @ (1/(Z - 1))
        return np.asscalar((-1/len(Y)) * dL)
    
    # Foward step
    def forward(self, X):
        V =  X @ self.W
        
        Z = self.sigmoid(V)
        return V,Z
    
    # Train neural network
    def train(self, X, Y):
        
        X = np.insert(X,0,1,axis=1) # inserting bias
        k = 0
        print("Epoch {}:".format(k))
        V, Z = self.forward(X)
        print("loss: {}".format(self.loss(Z,Y)))
        print("Predict: {}".format(self.predict(X,Y,bias=False)))
        loss = np.inf        
        # Perform a gradient descent algorithm
        while abs(loss) > 0.1 and k < 1000:
            k = k + 1
            #W_new = self.W - self.rate * self.gradient_loss(Z, Y) * X.T @ self.d_sigmoid(V)
            W_new = self.W + self.rate * np.linalg.pinv(self.H(X)) @ X.T @ (Y - Z)

            self.W = W_new 

            print("Epoch {}:".format(k))

            V, Z = self.forward(X)
            loss = self.loss(Z,Y)
            print(f"loss: {loss}")
            print(f"Predict: {self.predict(X,Y,bias=False)}")
    
    # Predict class of a sample
    def predict(self, X, Y, bias=True):
        if bias:
            X = np.insert(X,0,1,axis=1) # inserting bias
        Z = self.forward(X)[1].flatten().round()
        errors = 0
        for i,pred in enumerate(Z):
            if pred != Y[i]:
                errors += 1

        acc = 1 - (errors / len(Y))
        return acc       

In [514]:
nn = Perceptron(n_weights=30, rate=0.1)

In [515]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

data = load_breast_cancer()

y = data.target.reshape(-1,1)

X = data.data

X = preprocessing.scale(X)

In [516]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

nn.train(X_train, y_train)

Epoch 0:
loss: 0.7813963889553615
Predict: 0.5912087912087912
Epoch 1:
loss: 0.6286690780756534
Predict: 0.6527472527472528
Epoch 2:
loss: 0.5380026013938117
Predict: 0.6989010989010989
Epoch 3:
loss: 0.47544276485005044
Predict: 0.7538461538461538
Epoch 4:
loss: 0.42695532418970494
Predict: 0.7978021978021979
Epoch 5:
loss: 0.3874239903116193
Predict: 0.8307692307692307
Epoch 6:
loss: 0.35415862998954944
Predict: 0.8615384615384616
Epoch 7:
loss: 0.3255409452534318
Predict: 0.8945054945054945
Epoch 8:
loss: 0.3005182842736584
Predict: 0.9120879120879121
Epoch 9:
loss: 0.27836698430496726
Predict: 0.9208791208791209
Epoch 10:
loss: 0.25856837801841925
Predict: 0.9362637362637363
Epoch 11:
loss: 0.2407384631510371
Predict: 0.945054945054945
Epoch 12:
loss: 0.2245853465759263
Predict: 0.9516483516483516
Epoch 13:
loss: 0.20988198455293663
Predict: 0.9538461538461538
Epoch 14:
loss: 0.19644786472116094
Predict: 0.9626373626373627
Epoch 15:
loss: 0.18413630067786008
Predict: 0.967032967032

In [79]:
nn.predict(X_test, y_test)

0.9473684210526316

In [403]:
from sklearn.datasets import load_iris
data = load_iris()
X = data.data
y = data.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(len(y))
print(X.shape)

150
(150, 4)


In [507]:
class Dense(object):
    def __init__(self, units, n_weights, rate=0.1):
        self.units = units
        self.n_weights = n_weights
        self.rate = rate
        W = np.random.random((n_weights, 3))
        W = np.insert(W, 0, 1, axis=0) # inserting bias
        self.W = W
        
    # Activation function
    def softmax(self, x):
        exp = np.exp(x)
        return exp / np.sum(exp, axis=1)[:, None]
        
    def forward(self, X):
        f = X @ self.W
        probas = self.softmax(f)

        return f, probas
    
    def loss(self, y, probas):
        
        return -sum(np.einsum('ij,ij->i', y, np.log(probas + 1.e-10)))
    
    def gradient_loss(self, X, y, probas):
        return (1/y.shape[0]) * X.T @ (probas - y)
    
    def train(self, X, y):
        X = np.insert(X,0,1,axis=1) # inserting bias
        import pandas as pd
        y_h = np.array(pd.get_dummies(y_train)) # One hot encode target vector
        
        k = 0
        print("Epoch {}:".format(k))
        
        f, probas = self.forward(X)
        
        print(f"loss: {self.loss(y_h, probas)}")
        print(f"Predict: {self.predict(X, y, bias=False)}")
        loss = np.inf
              
        # Perform a gradient descent algorithm
        while abs(loss) > 0.1 and k < 500:
            k = k + 1
            
            W_new = self.W - self.rate * self.gradient_loss(X, y_h, probas)

            self.W = W_new 

            print("Epoch {}:".format(k))

            f, probas = self.forward(X)
            loss = self.loss(y_h,probas)
            
            print(f"loss: {loss}")
            print(f"Predict: {self.predict(X,y,bias=False)}")
        
        
    def predict(self, X, y, bias=True):
        if bias:
            X = np.insert(X,0,1,axis=1) # inserting bias
        probas = self.forward(X)[1]
        #print(probas)
        errors = 0
        for i,prob in enumerate(probas):
            pred = np.argmax(prob)
            if pred != y[i]:
                errors += 1

        acc = 1 - (errors / len(y))
        return acc       


In [508]:
dense = Dense(3, 4, 0.1)

In [509]:
dense.train(X_train, y_train)

Epoch 0:
loss: 287.3729543184365
Predict: 0.33333333333333337
Epoch 1:
loss: 210.9164154578986
Predict: 0.32499999999999996
Epoch 2:
loss: 146.88422817544395
Predict: 0.35
Epoch 3:
loss: 133.97330892661714
Predict: 0.32499999999999996
Epoch 4:
loss: 119.62265600892502
Predict: 0.6
Epoch 5:
loss: 111.9695887382155
Predict: 0.32499999999999996
Epoch 6:
loss: 105.35333309144137
Predict: 0.6083333333333334
Epoch 7:
loss: 100.29785769135201
Predict: 0.6583333333333333
Epoch 8:
loss: 95.67595697091737
Predict: 0.5833333333333333
Epoch 9:
loss: 92.7619179575935
Predict: 0.6583333333333333
Epoch 10:
loss: 89.41264696803034
Predict: 0.7083333333333333
Epoch 11:
loss: 88.33396338209691
Predict: 0.6583333333333333
Epoch 12:
loss: 85.67572915074967
Predict: 0.725
Epoch 13:
loss: 86.31641005873655
Predict: 0.6583333333333333
Epoch 14:
loss: 83.65372032047829
Predict: 0.6916666666666667
Epoch 15:
loss: 85.79502912372605
Predict: 0.6583333333333333
Epoch 16:
loss: 82.3681000849882
Predict: 0.675
Epoc

loss: 29.024510358366552
Predict: 0.975
Epoch 232:
loss: 28.97082092494462
Predict: 0.975
Epoch 233:
loss: 28.91740756563807
Predict: 0.975
Epoch 234:
loss: 28.864268062349176
Predict: 0.975
Epoch 235:
loss: 28.81140022291098
Predict: 0.975
Epoch 236:
loss: 28.758801880704983
Predict: 0.975
Epoch 237:
loss: 28.706470894173695
Predict: 0.975
Epoch 238:
loss: 28.6544051464335
Predict: 0.975
Epoch 239:
loss: 28.60260254483256
Predict: 0.975
Epoch 240:
loss: 28.55106102056811
Predict: 0.975
Epoch 241:
loss: 28.49977852827738
Predict: 0.975
Epoch 242:
loss: 28.448753045664898
Predict: 0.975
Epoch 243:
loss: 28.397982573118806
Predict: 0.975
Epoch 244:
loss: 28.34746513335096
Predict: 0.975
Epoch 245:
loss: 28.297198771034395
Predict: 0.975
Epoch 246:
loss: 28.247181552457093
Predict: 0.975
Epoch 247:
loss: 28.197411565177806
Predict: 0.975
Epoch 248:
loss: 28.14788691769377
Predict: 0.975
Epoch 249:
loss: 28.098605739112966
Predict: 0.975
Epoch 250:
loss: 28.049566178835484
Predict: 0.975
E

In [512]:
dense.predict(X_test, y_test)    

1.0